In [2]:
import pandas as pd
import numpy as np

In [2]:
patient_csv = pd.read_csv('common data/demographics.csv')

In [4]:
icd9_df = pd.read_csv('common data/all_icd_codes.csv')
icd9_df['icd9_codes'] = icd9_df['icd9_codes'].map(str)

In [5]:
merged_icd9_df = icd9_df.merge(patient_csv, how='inner', left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])

In [6]:
code_descriptions = pd.read_csv('../../bigdata/MIMIC/D_ICD_DIAGNOSES.csv')
def get_code_description(code) :
    coded =  list(code_descriptions[code_descriptions['ICD9_CODE'] == code]['LONG_TITLE'])
    if len(coded) == 0 :
        return 'ERROR ERROR'
    else :
        return coded[0]

In [7]:
def match_any(code_list, patterns) :
    return any([any([x.startswith(p) for p in patterns]) for x in code_list])

## Add Hypertension

In [8]:
merged_icd9_df['hypertension'] = merged_icd9_df['icd9_codes'].apply(lambda s : int(match_any(s.split(';'), ['401', '402', '403', '404', '405'])))

In [9]:
notes_df = pd.read_csv('common data/all_discharge_summaries_cleaned.csv.xz', compression='infer')

In [10]:
notes_df = notes_df.sort_values(by='description', ascending=False).reset_index(drop=True)

In [11]:
notes_df = notes_df.groupby(['subject_id', 'hadm_id'], as_index=False).agg({'text' : lambda s : " ".join(s)})

In [12]:
notes_merged_icd9 = merged_icd9_df.merge(notes_df, how='inner', left_on=['subject_id', 'hadm_id'], right_on=['subject_id', 'hadm_id'])

In [13]:
notes_merged_icd9.to_csv('Hypertension/data_with_notes.csv.xz', compression='xz')

In [14]:
notes_merged_icd9.drop(columns=['text']).to_csv('Hypertension/data_nonotes.csv')

In [3]:
code_descriptions = pd.read_csv('../../bigdata/MIMIC/D_ICD_DIAGNOSES.csv')
def get_code_description(code) :
    coded =  list(code_descriptions[code_descriptions['ICD9_CODE'] == code]['LONG_TITLE'])
    if len(coded) == 0 :
        return 'ERROR ERROR'
    else :
        return coded[0]

In [54]:
descs = list(code_descriptions['LONG_TITLE'])

In [55]:
from cleaners import cleaner_mimic

In [56]:
import re
def cleaner(text) :
    text = re.sub(r'([()\[\]])', r' \1 ', text)
    return cleaner_mimic(text)

In [57]:
descs = [cleaner(x).split(' ') for x in descs]

In [58]:
words = set([x for y in descs for x in y])

In [59]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/media/jainsarthak/data/projects/bigdata/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)

In [60]:
in_pre = 0
non = []
for w in list(words) :
    if w in model :
        in_pre += 1
    else :
        non.append(w)

In [ ]:
list(code_descriptions['ICD9_CODE'])